<a href="https://colab.research.google.com/github/hrsh668/ml/blob/main/SVM_for_image_classification_of_cifar_10_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if len(device_name) > 0:
    print("Found GPU at: {}".format(device_name))
else:
    device_name = "/device:CPU:0"
    print("No GPU, using {}.".format(device_name))

In [ ]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from scipy.stats import reciprocal, uniform
# Load the CIFAR-10 dataset (replace this with your method of loading CIFAR-10)

from tensorflow.keras.datasets import cifar10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# Flatten the images into feature vectors
X_train_flat = X_train.reshape(X_train.shape[0], -1)
X_test_flat = X_test.reshape(X_test.shape[0], -1)

# Normalize pixel values to the range [0, 1]
X_train_flat = X_train_flat.astype('float32') / 255.0
X_test_flat = X_test_flat.astype('float32') / 255.0

# Flatten labels (since SVM in scikit-learn expects 1D array as target)
y_train_flat = y_train.flatten()
y_test_flat = y_test.flatten()
fraction_to_keep = 0.1

# Split the dataset into training and testing sets
X_train, X_val, y_train, y_val = train_test_split(X_train_flat, y_train_flat, test_size=0.001, train_size = fraction_to_keep, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test_flat = scaler.transform(X_test_flat)

In [ ]:
# Create an SVM classifier
svm = SVC()

# Define the parameter grid to search
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'gamma': [0.01, 0.1, 1]
}

In [ ]:
with tf.device(device_name):
    grid_search = GridSearchCV(svm, param_grid, cv=5, scoring='accuracy')

  # Fit the randomized search to the data
    grid_search.fit(X_train, y_train)

  # Print the best parameters and the corresponding accuracy
    print("Best Parameters: ", grid_search.best_params_)
    print("Best Cross-Validated Accuracy: {:.2f}".format(grid_search.best_score_))

  # Evaluate the best model on the test set
    best_svm = grid_search.best_estimator_
    test_accuracy = best_svm.score(X_test_flat, y_test)
    print("Test Set Accuracy: {:.2f}".format(test_accuracy))